<a href="https://colab.research.google.com/github/nbrunella19/CIINTI/blob/main/CIINTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/INTI.svg/1200px-INTI.svg.png" width="100" alt="10">
Bienvenido a la aplicación de Análisis de medición de resistencia eléctrica del departamento de patrones cuánticos de INTI.

La funcionalidad del software es tomar los archivos recibidos desde la aplicación del 6010D junto a las mediciones de temperatura, mostrar y determinar el resultado de las mediciones.

Programación de funciones __2024__: [Nicolás Brunella](mailto:nbrunella@inti.gob.ar)

Colaboración y modificaciones __2024__: [Mariano Real](mailto:mreal@inti.gob.ar)

# Inicialización


## Librerias

In [59]:
import io                                   # Uso en funciones de Carga
import glob                                 # Uso en funciones de Carga
import os                                   # Uso en funciones de Carga
import sys                                  # Para explicar error y excepciones
import re                                   # Uso en funciones de Carga
import traceback                            # Uso en funciones de Carga
import cmath                                # Uso cmath para calcular raices
import copy
import numpy as np                          # Uso análisis de archivo de medición de resistencia
import pandas as pd                         # Uso análisis de archivo de medición de resistencia
import ipywidgets as widgets                # Uso en entorno gráfico
import matplotlib.pyplot as plt             # Uso en entorno gráfico
from io import StringIO                     # Uso análisis de archivo de medición de temperatura
from datetime import datetime               # Uso análisis de archivo de medición de temperatura
from google.colab import files              # Uso en funciones de Carga
from IPython.display import FileLink        # Uso en entorno gráfico
from IPython.display import display, HTML   # Uso en entorno gráfico
from IPython.display import clear_output    # Uso en entorno gráfico

## Variables Globales

In [60]:
flag_lmea=0
flag_ldat=0

num_measure_est=10                # Valor default de valores para estadística

Standar_1=[0.9999999,-0.00000007,-0.000000034,23.0] #Ro,alfa,beta,To
Rs_R0=0.9999999
Rs_alpha=-0.00000007
Rs_beta=-0.000000034
Rs_T=23.0


flag_En_Calc=0
flag_En_Graf=0
flag_Ch_Drop=0
index=1

## Funciones Generales

In [61]:
def counter_different_values(lista):
    '''
    Cuenta las diferentes corrientes de medición que se realizaron
    Input: Lista
    Output: int
    '''
    different_values = set(lista)
    different_quantity = len(different_values)
    return different_quantity


In [62]:
def solve_quadratic(a, b, c):
    '''
    Esta ecuación resuelve la cuadrática asumiendo que
    ax**2+bx+c=0
    input: coeficientes reales
    output: raices
    '''
    # Calculate the discriminant
    discriminant = cmath.sqrt(b**2 - 4*a*c)

    # Calculate the solutions
    root1 = (-b + discriminant) / (2*a)
    root2 = (-b - discriminant) / (2*a)

    root1 = root1.real if abs(root1.imag) < 1e-10 else float('nan')
    root2 = root2.real if abs(root2.imag) < 1e-10 else float('nan')

    return float(root1), float(root2)

In [63]:
def openFile(path2file):
    '''
    Abre el archivo, devuelve las líneas del archivo path2file.
    Asume que el archivo es un .mea
    Input= path
    output = lineas de un archivo (strings)
    '''
    try:
        with open(path2file, mode='r') as f:  # to ensure it closes correclty
            lines = f.readlines()
            return lines
    except Exception as exception:
        print("There was a problem opening {}".format(path2file))
        print(exception)
        raise

In [64]:
def openFile_T(path2file):
    '''
    Abre el archivo, devuelve las líneas del archivo path2file.
    Asume que el archivo es un .dat
    Input= path
    output = lineas de un archivo (strings)
    '''
    try:
        with open(path2file, mode='r', encoding='utf-8', errors='ignore') as g:  # to ensure it closes correclty
            lines = g.readlines()
            return lines
    except Exception as exception:
        print("There was a problem opening {}".format(path2file))
        print(exception)
        raise

In [65]:
def read_mea_file(lines_of_mea_file, ln_preface = 81):
    '''
    Esta función toma lo que se obtuvo de openFile, lo troza y ordena en listas
    Inputs:
        lines_of_mea_file: Lista de lineas de string obtenidas de openFile()
        ln_preface(optional, default=  81): Número de lineas a saltear donde empiezan los datos de medición.
    Outputs:
        qlistInit: Lista que contiene todos los datos técnicos de la medición
        qlist: Lista que contiene las mediciones. Por ej. lista[2]= Medición 3
        qlistFinal: Lista que contiene un resumen de la medición y fecha de realización
    Para el porceso es recomendable que todos los archivos estén juntos en el mismo archivo.
    Notar que no importa si las mediciones tienes diferentes cantidad de puntos.
    '''
    chunks = []
    qlist = []
    chunksInit =[]
    qlistInit = []
    chunksFinal = []
    qlistFinal = []
    nn = 0
    Quantity_Measurements = 0
    ii = 0
    for line in lines_of_mea_file:
        if (nn>ln_preface and line != '\n' and ii == 0):
            chunks.append(line)
            nn += 1
        elif (nn<ln_preface):
            chunksInit.append(line)
            nn +=1
        elif (nn>ln_preface and line == '\n'):
            ii = 1
            nn += 1
        elif (nn>ln_preface and ii == 1 and line != '***\n'):
            chunksFinal.append(line)
            nn += 1
        elif (line =='***\n'):
            ii = 0
            Quantity_Measurements += 1
            nn = 0
            qlist.append(chunks)
            qlistInit.append(chunksInit)
            qlistFinal.append(chunksFinal)
            chunks = []
            chunksInit = []
            chunksFinal = []
        else:
            nn += 1
    return qlistInit, qlist, qlistFinal,Quantity_Measurements

In [66]:
def select_arch_dat():
    '''
    Función para seleccionar un archivo
    Input: nada
    Output: FileLink/Path
    '''
    file_path_i=''
    uploaded = files.upload()                   # Muestra un cuadro de diálogo para cargar un archivo

    ruta_al_archivo = list(uploaded.keys())[0]  # Obtiene la ruta del archivo cargado

    _, extension = os.path.splitext(ruta_al_archivo)
    if extension.lower() == '.dat':
      file_path_i = FileLink(ruta_al_archivo)     # Crea un enlace para abrir el archivo
    else:
      file_path_i=''

    return (file_path_i)                        # Muestra el enlace

In [67]:
def select_arch_mea():
    '''
    Función para seleccionar un archivo
    Input: nada
    Output: FileLink/Path
    '''
    file_path_i=''
    uploaded = files.upload()                   # Muestra un cuadro de diálogo para cargar un archivo

    ruta_al_archivo = list(uploaded.keys())[0]  # Obtiene la ruta del archivo cargado

    _, extension = os.path.splitext(ruta_al_archivo)
    if extension.lower() == '.mea':
      file_path_i = FileLink(ruta_al_archivo)     # Crea un enlace para abrir el archivo
    else:
      file_path_i=''

    return (file_path_i)                        # Muestra el enlace

In [68]:
def get_Date_num(Date_aux):
  '''
  Toma una linea de texto en con la fecha y horaria y devuelve
  un string con la fecha expresada en dd/mm/aaaa y un valor numerico representativo
  '''
  Date_1=Date_aux.split()
  Date=Date_1[0]
  Time=Date_1[1]

  Date_s=Date.split("/")
  Day=Date_s[0]
  Mon=Date_s[1]
  Yea=Date_s[2]

  Time_s=Time.split(":")
  Hour=Time_s[0]
  Minu=Time_s[1]
  Seco=Time_s[2]

  auxday=int(Day)
  auxmon=int(Mon)
  if auxday<10:
    Day="0"+Day
  if auxmon<10:
    Mon="0"+Mon

  Date_num=int(Yea+Mon+Day+Hour+Minu+Seco)

  return Date,Date_num,Time


In [69]:
Sonda = 0
datos_Rosemoun= [100,0.004,-7.5*10**(-7)]                     # Valores inventados
datos_PT10002= [99.942247,0.0039253221,-7.5009932*10**(-7)]
SondaList=['STRP Rosemount','LPC PT100-02','STRP Rosemount y LPC PT100-02']

SDic = {SondaList[0]: datos_Rosemoun, SondaList[1]: datos_PT10002}
Files = ['R_0', 'Alpha', 'Beta']
df_s = pd.DataFrame(SDic, index=Files)


In [70]:
# Crear widgets
button_mea = widgets.Button(description="Cargar *.mea")
button_dat = widgets.Button(description="Cargar *.dat")
button_est_ok = widgets.Button(description="Setear")
input_text_est = widgets.Text(value= str(num_measure_est), description='Muestras:')


Sonda_dropdown= widgets.Dropdown(
    options=SondaList,
    value=SondaList[Sonda],
    description='Sonda:')

def handle_dropdown_Sonda(change):
    global Sonda, Sonda_name
    Sonda_name = change.new
    if(Sonda_name=='STRP Rosemount'): Sonda=0
    if(Sonda_name=='LPC PT100-02'): Sonda=1
    if(Sonda_name=='STRP Rosemount y LPC PT100-02'): Sonda=2


Sonda_dropdown.observe(handle_dropdown_Sonda, names='value')

def on_button_mea_click(b):                      #Carga de .mea
    global flag_lmea,file_path
    file_path= str(select_arch_mea())
    if file_path=='':
      Ejecutar_para_configurar()
      print("La extensión del archivo no es la correcta.Debe ser .mea")


def on_button_dat_click(b):                      #Carga de .dat
    global flag_ldat,file_path_t
    file_path_t= str(select_arch_dat())
    if file_path_t=='':
      Ejecutar_para_configurar()
      print("La extensión del archivo no es la correcta.Debe ser .dat")

def on_button_set1(b):
      global num_measure_est
      num_measure_aux=(input_text_est.value).strip()
      if num_measure_aux.isdigit():
          num_measure_est=int(num_measure_aux)
          display("Muestras tomadas para estadística: "+num_measure_aux)
          display("Nota: Si la cantidad de muestras excede la cantidad de muestras por medición se tomarán el total de estas")
      else:
          Ejecutar_para_configurar()
          display("El valor introducido no es numérico. Por favor intentarlo nuevamente")

# Configurar la función a llamar cuando se hace clic en el botón
button_mea.on_click(on_button_mea_click)
button_dat.on_click(on_button_dat_click)
button_est_ok.on_click(on_button_set1)

def Ejecutar_para_configurar():
    clear_output()
    display(button_mea)
    display(button_dat)
    display(input_text_est)
    display(button_est_ok)
    display(Sonda_dropdown)

#Carga y configuración


In [71]:
Ejecutar_para_configurar()

Button(description='Cargar *.mea', style=ButtonStyle())

Button(description='Cargar *.dat', style=ButtonStyle())

Text(value='10', description='Muestras:')

Button(description='Setear', style=ButtonStyle())

Dropdown(description='Sonda:', options=('STRP Rosemount', 'LPC PT100-02', 'STRP Rosemount y LPC PT100-02'), va…

Saving 2024-02-01 LBP161.mea to 2024-02-01 LBP161 (1).mea


Saving 2024-02-01LBP161.DAT to 2024-02-01LBP161 (1).DAT


'Muestras tomadas para estadística: 10'

'Nota: Si la cantidad de muestras excede la cantidad de muestras por medición se tomarán el total de estas'

# Ejecutar para análisis de archivos

## Análisis de archivo de medición: *.mea

In [72]:
lineas = openFile(file_path)
qlistInit_aux, qlist_aux, qlistFinal_aux,Quantity_Measurements_aux= read_mea_file(lineas, ln_preface = 81)


In [73]:
qlistFinal=[]
qlistInit=[]
qlist=[]
Quantity_Measurements=0

for i in range(Quantity_Measurements_aux):                      # Solo mediciones buenas proporcionan 12 líneas en qListFinal_aux
  if len(qlistFinal_aux[i])==12:                                # Si es así la agrego
      qlistInit.append(qlistInit_aux[i])
      qlist.append(qlist_aux[i])
      qlistFinal.append(qlistFinal_aux[i])

Quantity_Measurements=len(qlist)                                # Variable muy importante

In [74]:
def create_df_from_list(qlist, n,num_measure_est):
    '''
    Crea un DataFrame desde una lista proveniente del .mea
    Input: listado,número a iterar
    Output: Dataframe
    '''
    df_aux = pd.DataFrame([sub.split(';') for sub in qlist[n]])
    size_df_m=df_aux.shape[0]
    if size_df_m<num_measure_est:                                     # Si el número es mayor que el medido se toma toda el largo de la medición
      num_measure_est=size_df_m
    df = df_aux.tail(num_measure_est)                                 # Recorta la lista a num_measure_est de largo
    df.columns = ['Ratio', 'Measure', 'Time']
    return df


In [75]:
df_m = [create_df_from_list(qlist, i,num_measure_est) for i in range(Quantity_Measurements)]          # Crea una lista de Dataframe

for i in range(Quantity_Measurements):
    df_m[i]['Time'] = df_m[i]['Time'].str.strip()

In [76]:
measure_f=[]                                                            #Lista de mediciones en formato float
time_h=[]                                                               #Lista de hora en formato "H":"M":"S"
ratio=[]
for i in range(Quantity_Measurements):                                                # Genero un listado de listas del tipo float
      measure_f.append([float(numero) for numero in df_m[i]["Measure"]])
      ratio.append([float(numero) for numero in df_m[i]["Ratio"]])
      time_h.append(df_m[i]["Time"])

In [77]:
                                              #Listados de los parámetros de las mediciones
iList=[0]*Quantity_Measurements               #Listado de corrientes
RsList=[0]*Quantity_Measurements              #Listado de patrones
RxList=[0]*Quantity_Measurements              #Listado de Mesurandos
MvList=[0]*Quantity_Measurements              #Listado de Mediciones medias
StList=[0]*Quantity_Measurements              #Listado de desvios
StartDateList=[0]*Quantity_Measurements       #Listado de fecha de inicio
StartDateNumList=[0]*Quantity_Measurements    #Listado de fecha y horario de inicio formato entero
StopDateList=[0]*Quantity_Measurements        #Listado de fecha de finalización
StopDateNumList=[0]*Quantity_Measurements     #Listado de fecha y horario de finalización formato entero
StartHourList=[0]*Quantity_Measurements
StopHourList=[0]*Quantity_Measurements
Rs_modelList=[0]*Quantity_Measurements
Rx_modelList=[0]*Quantity_Measurements

for i in range(Quantity_Measurements):
  StartDate_aux= qlistInit[i][13].strip()
  StartDateList[i],StartDateNumList[i],StartHourList[i]= get_Date_num(StartDate_aux)  # Obtengo un valor tipo string para presentar y otro int para operar

  StopDate_aux= qlistFinal[i][11].strip()
  StopDateList[i],StopDateNumList[i],StopHourList[i]=get_Date_num(StopDate_aux)       #Idem

  iList[i]=  qlistInit[i][22].strip()         #Línea donde se guarda la corriente Rx
  RsList[i]= qlistInit[i][33].strip()         #Línea donde se guarda la que valor nominal de patrón se usó
  Rs_modelList[i]= qlistInit[i][39].strip()   #Línea donde se guarda el modelo del patrón
  RxList[i]= qlistInit[i][57].strip()         #Línea donde se guarda la que valor nominal de patrón se usó
  Rx_modelList[i]= qlistInit[i][64].strip()   #Línea donde se guarda el modelo del resistor a medir


In [78]:
amList=[0]*Quantity_Measurements
gammaList=[0]*Quantity_Measurements

for i in range(Quantity_Measurements):
  ratio_aux=np.mean(ratio[i])                                           # Valor promedio de gamma (Rx/Rs) medido por medición
  gammaList[i]=ratio_aux


## Análisis de archivo de medición: *.dat

In [79]:
lineas_t = openFile_T(file_path_t)        #

In [80]:
Measures_of_datlist=[]

for line in lineas_t:                     # Lectura de .dat
  line_header= line[:6]                   # line_header detecta el encabezado
  line_sensor= line[:8]                   # line_sensor me permite saber que sonda es
  if(line_header!="sensor"):
    if ":" in line:
      Measures_of_datlist.append(line)    # Agrega a la lista la info



df_t = pd.DataFrame([linea.split(",") for linea in Measures_of_datlist], columns=['Date Hour T', 'Mean S1', 'Mean S2', 'Time T'])

size_df_t=df_t.shape[0]                   # Variable que me dice la cantidad de mediciones de temperatura que hay

for i in range(size_df_t):
    df_t['Time T'] = df_t['Time T'].str.strip()

df_t['Time T'] = df_t['Time T'].astype(int)
df_t['Mean S1'] = df_t['Mean S1'].astype(float)
df_t['Mean S2'] = df_t['Mean S2'].astype(float)


In [81]:
Val_Sonda1_Ohm=[0]*size_df_t
Val_Sonda2_Ohm=[0]*size_df_t

for i in range(size_df_t-1):
  Val_Sonda1_Ohm[i]=df_t.loc[i, 'Mean S1'].astype(float)          #Rosemount
  Val_Sonda2_Ohm[i]=df_t.loc[i, 'Mean S2'].astype(float)          #LPC-PT100

In [82]:
def Temp_ave (R_T,Sonda_T):
  '''
  Función que calcula temperatura promedio del listado de entrada
  Input = promedio de mediciones en Ohm
  Output= promedio de temperatura en grados centígrados
  '''
  Tfinal=0
  if(Sonda_T==0):
    SColumn='STRP Rosemount'
  if(Sonda_T==1):
    SColumn='LPC PT100-02'

  R_0 = float(df_s.iloc[0, df_s.columns.get_loc(SColumn)])
  alpha=float(df_s.iloc[1, df_s.columns.get_loc(SColumn)])
  beta =float(df_s.iloc[2, df_s.columns.get_loc(SColumn)])

  c = R_0 - R_T
  b = R_0*alpha
  a = R_0*beta

  T1,T2 = solve_quadratic(a, b, c)

  if T2>0 and T2<35:
    Tfinal= T2
  elif T1>0 and T1<35:
    Tfinal= T1
  return Tfinal

In [83]:
def calc_temp_ave_List(index_T):
  '''
  Función que calcula temperatura promedio de una medición puntual
  Input = Número de Medición
  Output= promedio de temperatura en grados centígrados
  '''
  R_Tlist_S1=[]
  R_Tlist_S2=[]

  Start_Mean= StartDateNumList[index_T]
  Stop_Mean = StopDateNumList[index_T]

  for i in range(size_df_t):

    time_mean_temp = int(df_t.iloc[i, df_t.columns.get_loc('Time T')])

    if time_mean_temp>=Start_Mean and time_mean_temp<=Stop_Mean:
        R_Tlist_S1.append(Val_Sonda1_Ohm[i])
        R_Tlist_S2.append(Val_Sonda2_Ohm[i])

  if R_Tlist_S1:
    T_aux_1=Temp_ave(np.mean(R_Tlist_S1),0)
  else:
    T_aux_1=0
  if R_Tlist_S2:
    T_aux_2=Temp_ave(np.mean(R_Tlist_S2),1)
  else:
    T_aux_2=0
  return T_aux_1,T_aux_2

In [84]:
TempList_S1=[0]*Quantity_Measurements                                  #Temp promedio por medición (Rosemount)
TempListString_S1=[""]*Quantity_Measurements                           #Idem en formato string
TempList_S2=[0]*Quantity_Measurements                                  #Temp promedio por medición (PT100)
TempListString_S2=[""]*Quantity_Measurements                           #Idem en formato string


for i in range(Quantity_Measurements):                                 #Tomo valores
  TempList_S1[i],TempList_S2[i]= calc_temp_ave_List(i)

  if TempList_S1[i]==0:                                                #Si dan 0 es que la medición no fue tomada
    Temp_aux= "Temperatura no Medida"                                  #Puede existir otra condición
  else:
    Temp_aux=str(round(TempList_S1[i],2))
  TempListString_S1[i]=Temp_aux

  if TempList_S2[i]==0:
    Temp_aux="Temperatura no medida"
  else:
    Temp_aux=str(round(TempList_S2[i],2))
  TempListString_S2[i]=Temp_aux



In [85]:
measure_fixed_TList=[]                                                        #Listado de listas que permiten el cálculo de la resistencia corregida
aux_measure_fixed_T=[0]*num_measure_est
measure_fixed_T=[0]*Quantity_Measurements

if Sonda==0:tParam =TempList_S1
if Sonda==1:tParam =TempList_S2

for i in range(Quantity_Measurements):                                          # Bucle que forma un listado de listas con los resultados de resistencia corregidos por temperatura
  T=tParam[i]
  a_sd = Rs_alpha*(T-Rs_T)
  b_sd = (Rs_beta*(T-Rs_T))**2
  for j in range(num_measure_est):

    gamma_sd=ratio[i][j]
    aux_measure_fixed_T[j]= gamma_sd* Rs_R0* float((1+a_sd+b_sd))

  measure_fixed_TList.append(copy.deepcopy(aux_measure_fixed_T))                # Usa copy porque sino agregaba solo último listado muchas veces


In [86]:
stdList_T=[0]*Quantity_Measurements                                             # Correcciones por temperaturas
stdList_T_ppm=[0]*Quantity_Measurements

for i in range(Quantity_Measurements):
    measure_fixed_T[i]=np.mean(measure_fixed_TList[i])                        # Valor resistencia promedio por medición corregida en temperatura
    stdList_T[i]=np.std(measure_fixed_TList[i])                               # Valor de desviación promedio por medición corregida en temperatura
    stdList_T[i]=np.std(measure_fixed_TList[i])
    stdList_T_ppm[i]=str(round((stdList_T[i]/measure_fixed_T[i])*10**6,4))

am_full_str=str(round(np.mean(measure_fixed_T),9))
sd_full_str=str(round(np.std(measure_fixed_T),9))

t_x_full= [f"Medición {i+1}" for i in range(Quantity_Measurements)]             # Listado con la cantidad de mediciones para luego graficar


In [87]:
if Sonda==2:
          df_show = pd.DataFrame({'Medición': t_x_full,'Resistencia Media (Ω)':measure_fixed_T,'Desvío Estandar(ppm)':stdList_T_ppm, 'Temp. Medida con Rosemount (°C)': TempListString_S1,'Temp. Medida con LPC PT100-02 (°C)': TempListString_S2, 'Fecha de inicio' :StartDateList,'Horario de Inicio': StartHourList,'Fecha de Finalización' :StopDateList,'Horario de Finalización': StopHourList})
elif Sonda==1:
          df_show= pd.DataFrame({'Medición': t_x_full,'Resistencia Media (Ω)':measure_fixed_T,'Desvío Estandar(ppm)':stdList_T_ppm, 'Temp. Medida con LPC PT100-02  (°C)': TempListString_S2, 'Fecha de inicio' :StartDateList,'Horario de Inicio': StartHourList,'Fecha de Finalización' :StopDateList,'Horario de Finalización': StopHourList})
elif Sonda==0:
          df_show= pd.DataFrame({'Medición': t_x_full,'Resistencia Media (Ω)':measure_fixed_T,'Desvío Estandar(ppm)':stdList_T_ppm, 'Temp. Medida con Rosemount (°C)': TempListString_S1, 'Fecha de inicio' :StartDateList,'Horario de Inicio': StartHourList,'Fecha de Finalización' :StopDateList,'Horario de Finalización': StopHourList})



## Configuración de analizador


In [88]:
##################################################################################################################################
counter_m = []
for i in range(Quantity_Measurements):                             # Genera un listado en función de cuantas mediciones se leyeron
    counter_m.append(i+1)                                          # para que se muestre en la lista desplegable (dropdown)

menu_desplegable = widgets.Dropdown(                               # Creación widget de dropdown
    options=counter_m,
    value=counter_m[0],
    description='Medición:')


def handle_dropdown_change(change):                                # Función de cambio del dropdown
    global index
    if flag_Ch_Drop==0:
      index = change.new

                                                                   # Crear widgets de texto
output_text0  = widgets.Label(value='')
output_text1  = widgets.Label(value='')
output_text2  = widgets.Label(value='')
output_text3  = widgets.Label(value='')
output_text4  = widgets.Label(value='')
output_text5  = widgets.Label(value='')
output_text6  = widgets.Label(value='')
output_text7  = widgets.Label(value='')
output_text8  = widgets.Label(value='')
output_text9  = widgets.Label(value ="Fecha de inicio :"       + StartDateList[0] + "  Hora: "+StartHourList[0])
output_text10 = widgets.Label(value ="Fecha de finalización :" + StopDateList[Quantity_Measurements-1] + "  Hora: "+StopHourList[Quantity_Measurements-1])
output_text11 = widgets.Label(value ="Resistor Incógnito :"    + RxList[0] + "Ω" )
output_text14 = widgets.Label(value ="Marca/Modelo :"          + Rx_modelList[0])
output_text12 = widgets.Label(value ="Resistor Patrón :"       + RsList[0] + "Ω")
output_text13 = widgets.Label(value ="Marca/Modelo :"          + Rs_modelList[0] + "Ω")


def on_button_calcu_click(b):
  global flag_En_Calc,index,TempListString_S1,Sonda
  if flag_En_Calc==0:
      Show_Info1(index)
      flag_En_Calc=1

def Show_Info1(index_s):
      output_text0.value    = "Datos de la medición "           +str(index_s)
      output_text1.value    = "Resistencia Promedio: "          +str(measure_fixed_T[index_s-1]) + "Ω"
      output_text2.value    = "Desvío Estándar: "               + str(stdList_T[index_s-1])
      if(Sonda==0):
        output_text3.value  = "Temperatura media (Rosemount): " + TempListString_S1[index_s-1]  +"°C"
      if(Sonda==1):
        output_text3.value  = "Temperatura media (LPC-PT100): " + TempListString_S2[index_s-1]  +"°C"
      output_text4.value    = "Resistor Estándar (Rs): "        + RsList[index_s-1] +"Ω"
      output_text5.value    = "Valor nominal (Rx): "            + RxList[index_s-1] +"Ω"
      output_text6.value    = "Corriente Ix para ensayo: "      + iList[index_s-1]  +"A"
      display(df_m[index_s-1])

def on_button_clear_click(b):
    global index,flag_En_Calc, flag_En_Graf,flag_Ch_Drop
    output_text0.value = ''
    output_text1.value = ''
    output_text2.value = ''
    output_text3.value = ''
    output_text4.value = ''
    output_text5.value = ''
    output_text6.value = ''
    output_text7.value = ''
    output_text8.value = ''
    output_text9.value = ''
    output_text10.value = ''
    flag_En_Calc=0
    flag_En_Graf=0
    flag_Ch_Drop=0
    index=1
    menu_desplegable.value =index
    clear_output()
    display(tab)

def on_button_grafi_click(b):
    global index, flag_En_Graf
    if flag_En_Graf==0:
      t_x = time_h[index]
      r_y = measure_f[index]
      r_y_array = np.array(r_y, dtype=float)
      r_y_array = r_y_array - r_y_array.mean()
      plt.figure(figsize=(15,6))
      plt.xticks(rotation=45)
      display(plt.plot(t_x, r_y_array))
      flag_En_Graf=1

def on_button_show_full_click(b):
      global t_x_full , measure_fixed_T,am_full,sd_full
      output_text7.value = "Resistencia Promedio total: " + am_full_str + "Ω"
      output_text8.value = "Desvío estándar total: "      + sd_full_str
      r_y = measure_fixed_T
      r_y_array = np.array(r_y, dtype=float)
      r_y_array = r_y_array - r_y_array.mean()
      plt.figure(figsize=(15,6))
      plt.xticks(rotation=45)
      display(plt.plot(t_x_full, r_y_array))
      display(df_show)

def show_menu_1():
    display(output_text9)
    display(output_text10)
    display(output_text11)
    display(output_text14)
    display(output_text12)
    display(output_text13)
    display(button_show_full)
    display(output_text4)
    display(output_text5)
    display(output_text6)
    display(button_clear)
    display(output_text7)
    display(output_text8)


def show_menu_2():
    display(menu_desplegable)
    display(button_calcu)
    display(button_grafi)
    display(button_clear)
    display(output_text0)
    display(output_text1)
    display(output_text2)
    display(output_text3)
    display(output_text4)
    display(output_text5)
    display(output_text6)

def Mostrar_Resultados():
  display(tab)

button_calcu = widgets.Button(description="Calcular")                       #Tab2
button_clear = widgets.Button(description="Limpiar")                        #Tab1 y Tab2
button_grafi = widgets.Button(description="Graficar")                       #Tab2
button_show_full = widgets.Button(description="Ver resultados global")      #Tab1

# Configurar la función a llamar cuando se hace clic en el botón
button_calcu.on_click(on_button_calcu_click)
button_clear.on_click(on_button_clear_click)
button_grafi.on_click(on_button_grafi_click)
button_show_full.on_click(on_button_show_full_click)

menu_desplegable.observe(handle_dropdown_change, names='value')

################################################# Crear pestañas  ####################################################################

tab1 = widgets.Output()
tab2 = widgets.Output()

tab = widgets.Tab(children=[tab1, tab2])

# Agregar etiquetas a las pestañas
tab.set_title(0, 'Análisis General')
tab.set_title(1, 'Análisis por Medición')# Contenido de la primera pestaña
with tab1:
    show_menu_1()
# Contenido de la segunda pestaña
with tab2:
    show_menu_2()


############################################################################################################################################


# Analizador

In [ ]:
Mostrar_Resultados()
